##### Learned a lot from excellent others' notebooks.
##### Please feel free to leave an upvote if you find it helpful.
##### It will be my huge motivation. Appreciate it!

#### ideas have been testified
#### 1. performed feature engineering by aggregation method >> the most effective method
#### 2. exhausted popular gbdt algorithm: lightGBM, CatBoost, and XGBoost >> current best, catboost
#### 3. created a pipeline to incorporate both scalers and the classifier
#### 4. applied predict_prob rather than predict to output probabilites >> turns out to be effective

#### ideas yet to be testified
#### 1. hyperparameter tuning >> ongoing
#### 2. stackCVClassifier >> not quite works
#### 3. lagged features >> imrpoved a tiny bit
#### 4. LSTM models >> ongoing

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

import lightgbm as lgb
import catboost as cgb

from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_objective, plot_histogram

import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2022/train.csv')
df_train_labels = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2022/train_labels.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2022/test.csv')
df_smpl = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2022/sample_submission.csv')

In [ ]:
df_train_merged = df_train.merge(df_train_labels, on = 'sequence', how = 'left')
df_train_test = pd.concat([df_train_merged, df_test])

In [ ]:
# check the feature of subject

# each squence correspons to only one subject
# but each subject does not correspons to only one squence

plt.figure(figsize=(16, 5))

plt.bar(df_train_test.subject.value_counts().index,
            df_train_test.subject.value_counts())

#### no discernible pattern for the feature of subject

In [ ]:
# check correlation of each pair of features

fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(df_train_test.corr(),
            linewidths=0.1,
            annot=True, fmt='.2f', cmap='coolwarm', annot_kws={'size': 12},
            mask=np.triu(df_train_test.corr()))

In [ ]:
# check outliers
df_sensor = df_train_test.iloc[:,3: -1]

fig, axs = plt.subplots(4, 4, figsize=(20, 12))

for ax, sensor in zip(axs.flat, range(13)):
    label = format(sensor, '02')
    sensor_label = ('sensor_%s' % (label))
    ax.boxplot(df_sensor[sensor_label], vert=False)
    ax.set_title(sensor_label)

fig.tight_layout()

In [ ]:
# check the data distribution
fig, axs = plt.subplots(4, 4, figsize=(20, 12))

for ax, sensor in zip(axs.flat, range(13)):
    label = format(sensor, '02')
    sensor_label = ('sensor_%s' % (label))
    ax.hist(
        df_sensor[sensor_label], bins=100,
        range=(df_sensor[sensor_label].quantile(0.05),
               df_sensor[sensor_label].quantile(0.95))
    )
    ax.set_title(sensor_label)

fig.tight_layout()

In [ ]:
# normality check

import statsmodels.api as sm

fig, axs = plt.subplots(4, 4, figsize=(20, 12))

for ax, sensor in zip(axs.flat, range(13)):
    label = format(sensor, '02')
    sensor_label = ('sensor_%s' % (label))
    sm.qqplot(
        df_sensor[sensor_label], line='q', ax=ax
    )
    ax.set_title(sensor_label)

fig.tight_layout()

In [ ]:
# figure out the influence of each sensor data

from sklearn.feature_selection import mutual_info_classif

mi_score = mutual_info_classif(
    df_train_merged.iloc[:, 3:-1], df_train_merged['state'])

fig, ax = plt.subplots(figsize=(16,4))

ax.bar(df_sensor.columns , height = mi_score)

#### use these sensor features directly results in quite low auc scores >> feature engineering is necessary

In [ ]:
# add lagged features
# for 5s and 15s >> not working
# 1s and 5s >> not working
# 1s and 10s >> works!

sensor_list = df_train_test.columns[3:-1].tolist()

for sensor in sensor_list:
    
    lag_col_1 = sensor + '_lag_1'
    lag_col_10 = sensor + '_lag_10'
    
    df_train_test[lag_col_1] = df_train_test[sensor].shift(1)
    df_train_test[lag_col_10] = df_train_test[sensor].shift(10)

In [ ]:
# aggreagation method

df_copy = pd.concat([df_train_labels, df_smpl], axis=0).drop('state', axis = 1)

# use basic statistics to perform feature engineering on sensor data
stat_features = ['mean', 'std', 'skew', 'var', 'mad', 'min', 'max', 'median']

for feature in stat_features:

    df_temp = df_train_test.groupby(['sequence', 'subject'], as_index=True).agg(
        feature).drop(['step', 'state'], axis=1)

    col_name = list()

    for previous_col_name in df_temp.columns.to_list():
        new_col_name = previous_col_name + '_' + feature
        col_name.append(new_col_name)

    df_temp.columns = col_name
    df_temp.reset_index(inplace=True)
    df_temp.drop('subject', axis=1, inplace=True)
    df_copy = df_copy.merge(df_temp, on='sequence')
    
df_copy = df_copy.merge(df_train_labels, on = 'sequence', how = 'outer')

In [ ]:
# data prep

train_engineered = df_copy[df_copy['state'].isna() == False]
test_engineered = df_copy[df_copy['state'].isna()]

X_train = train_engineered.drop(['state'], axis=1)
y_train = train_engineered['state']

X_test = test_engineered.drop(['state'], axis=1)

In [ ]:
# tuned cgb model

steps = []
steps.append(('minmax_scaler', MinMaxScaler()))
steps.append(('std_scaler', StandardScaler()))
steps.append(('model', cgb.CatBoostClassifier(
    bagging_temperature = 0.14741981195117174,
    border_count = 200,
    depth=4,
    iterations=461,
    l2_leaf_reg = 25,
    learning_rate = 0.22774253759183857,
    random_strength = 0.2881729869911725,
    scale_pos_weight = 0.32624480187205324,
    verbose=False
)))

tuned_pipe_cgb = Pipeline(steps)

cv = RepeatedStratifiedKFold(n_splits=10,n_repeats=3)

scores = cross_val_score(
    tuned_pipe_cgb,
    X_train,
    y_train,
    scoring='roc_auc',
    cv=cv,
    n_jobs=-1
)

print('mean score: %.3f' % scores.mean())
print('std: %.3f' % scores.std())

In [ ]:
tuned_pipe_cgb.fit(X_train, y_train)
pred = tuned_pipe_cgb.predict_proba(X_test) # use predict_prob instead of predict
df_smpl['state'] = pd.Series(pred[:, 1]) # pred per se is composed of two columns

df_smpl[['sequence','state']].to_csv('submission.csv', index=False)